## Instalação de bibliotecas

In [ ]:
!pip install sktime
!pip install prophet
!pip install tensorflow
!pip install scikit-learn
!pip install dask

## Importação das bibliotecas

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import dask
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from prophet import Prophet
import tensorflow as tf
import dask.dataframe as dd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Carregamento da base de dados e visualização dos dados de treino

In [ ]:
# Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho da base de dados de treino e teste
path_train = '/content/drive/MyDrive/predição_serie_temporal/DailyDelhiClimateTrain.csv'
path_test = '/content/drive/MyDrive/predição_serie_temporal/DailyDelhiClimateTest.csv'

# Carregar os dados de treino e teste
train_data = pd.read_csv(path_train)
test_data = pd.read_csv(path_test)

# Visualizar os dados de treino
train_data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [ ]:
y_train = train_data['meantemp']
print(type(y_train))  # Deve retornar <class 'pandas.core.series.Series'>

<class 'pandas.core.series.Series'>


## Preparação de Dados para Análise de Série Temporal e Separação de Conjuntos de Treino e Validação

In [ ]:
# Preparar os dados para a série temporal
# Check if 'date' is already the index
if 'date' not in train_data.columns:
  # train_data['date'] = pd.to_datetime(train_data['date']) # This line is not needed as 'date' is already the index
  pass # Do nothing if 'date' is not in columns
train_data = train_data # Do not reset index as 'date' is already the index

# Série temporal de interesse (temperatura)
y_train = train_data['meantemp']

# Separar dados de treino e validação
y_train, y_val = temporal_train_test_split(y_train, test_size=0.2)

## Preparação e Treinamento do Modelo Prophet para Previsão de Séries Temporais

In [ ]:
# Preparar os dados para o Prophet
prophet_data = train_data.reset_index().rename(columns={'date': 'ds', 'meantemp': 'y'})

# Treinar o modelo Prophet
prophet_model = Prophet()
prophet_model.fit(prophet_data)

# Fazer previsões
future_dates = prophet_model.make_future_dataframe(periods=len(test_data))
forecast_prophet = prophet_model.predict(future_dates)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjm5ztoiy/_d493ycb.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjm5ztoiy/5linx6am.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=12023', 'data', 'file=/tmp/tmpjm5ztoiy/_d493ycb.json', 'init=/tmp/tmpjm5ztoiy/5linx6am.json', 'output', 'file=/tmp/tmpjm5ztoiy/prophet_model3pi1r8ib/prophet_model-20241002175937.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:59:37 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:59:37 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


## Criação de Dataset e Treinamento de um Modelo LSTM para Previsão de Séries Temporais

In [ ]:
# Função para criar dataset para LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(train_data['meantemp'].values.reshape(-1, 1))

# Criar conjunto de treino para LSTM
time_step = 30
X_train, Y_train = create_dataset(scaled_data, time_step)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Construir o modelo LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
lstm_model.add(LSTM(50, return_sequences=False))
lstm_model.add(Dense(25))
lstm_model.add(Dense(1))

# Compilar o modelo
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
lstm_model.fit(X_train, Y_train, batch_size=1, epochs=1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1431/1431 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - loss: 0.0166


## Preparação de Dados de Teste e Previsão Usando o Modelo LSTM

In [ ]:
# Preparar os dados de teste
test_data_scaled = scaler.transform(test_data['meantemp'].values.reshape(-1, 1))
X_test, _ = create_dataset(test_data_scaled, time_step)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Fazer previsões com o modelo LSTM
predicted_lstm = lstm_model.predict(X_test)
predicted_lstm = scaler.inverse_transform(predicted_lstm)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step


## Função de Avaliação de Previsões e Comparação de Desempenho dos Modelos Prophet e LSTM

In [ ]:
# Função para avaliar previsões
def evaluate_predictions(true_values, predicted_values):
    # Ensure both arrays are 1D and have the same length
    true_values = true_values.ravel()
    predicted_values = predicted_values.ravel()
    min_len = min(len(true_values), len(predicted_values))
    true_values = true_values[:min_len]
    predicted_values = predicted_values[:min_len]
    mse = mean_squared_error(true_values, predicted_values)
    rmse = np.sqrt(mse)
    return {'MAE': mae, 'MSE': mse, 'RMSE': rmse}

# Avaliar previsões dos modelos
results_prophet = evaluate_predictions(test_data['meantemp'], forecast_prophet['yhat'][:len(test_data)])

# Ensure predicted_lstm and test_data['meantemp'] have the same length for evaluation
results_lstm = evaluate_predictions(test_data['meantemp'], predicted_lstm)

print(f"Resultados Prophet: {results_prophet}")
print(f"Resultados LSTM: {results_lstm}")

Resultados Prophet: {'MAE': 2.5550745559733143, 'MSE': 10.045910399651305, 'RMSE': 3.1695284191266224}
Resultados LSTM: {'MAE': 2.8103119346768404, 'MSE': 12.087575116620556, 'RMSE': 3.4767190160581793}


<ipython-input-15-ebc001a4c9b4>:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  true_values = true_values.ravel()
<ipython-input-15-ebc001a4c9b4>:5: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  predicted_values = predicted_values.ravel()
<ipython-input-15-ebc001a4c9b4>:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  true_values = true_values.ravel()


# Escolha da métrica

Escolhi o RMSE (Root Mean Squared Error) como uma das principais métricas de avaliação, pois penaliza erros maiores de forma mais acentuada do que o MAE. Isso é importante em predições de séries temporais, onde grandes desvios podem ter impacto significativo. RMSE também é amplamente utilizada em previsões de séries temporais e é fácil de interpretar, pois está na mesma escala dos dados originais.